In [ ]:
!git clone https://github.com/arcee-ai/mergekit.git



In [ ]:
%cd mergekit

In [ ]:
pip install -e .

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
merge_config = """
slices:
- sources:
  - model: teknium/OpenHermes-2.5-Mistral-7B
    layer_range:
    - 0
    - 32
  - model: Open-Orca/Mistral-7B-OpenOrca
    layer_range:
    - 0
    - 32
merge_method: slerp
base_model: Open-Orca/Mistral-7B-OpenOrca
parameters:
  t:
  - filter: self_attn
    value:
    - 0
    - 0.5
    - 0.3
    - 0.7
    - 1
  - filter: mlp
    value:
    - 1
    - 0.5
    - 0.7
    - 0.3
    - 0
  - value: 0.5
dtype: bfloat16
"""

with open('config.yaml', 'w') as f:
    f.write(merge_config)

In [ ]:
!mergekit-yaml config.yaml ./merge --copy-tokenizer --allow-crimes --out-shard-size 1B --trust-remote-code

In [ ]:
from huggingface_hub import HfApi

In [ ]:
api = HfApi(token="YOUR_TOKEN")

In [ ]:
username = "your_username"
MODEL_NAME = "orca-teknium-merge"

In [ ]:
api.create_repo(
    repo_id = f"{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    repo_id = f"{username}/{MODEL_NAME}",
    folder_path = "merge"
)

In [ ]:
pip install -U bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "your_username/orca-teknium-merge"

# Enable quantization with bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def generate_response(user_input):
    prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction:
{user_input}

### Response:
"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=512, 
        num_beams=10, 
        early_stopping=True, 
        no_repeat_ngram_size=2
    )
    
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    return response

# Chat loop
if __name__ == "__main__":
    print("Chat with the AI! Type 'exit' to quit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break
        response = generate_response(user_input)
        print("AI:", response)